In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv(r'../input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv(r'../input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.nunique()

checking if any null values

In [ ]:
train.isnull().sum()

looks like there is none.
let's see what types of data present in data:

In [ ]:
train.dtypes

using **label encoder** for convert object datatypes:

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in range(19):
    le.fit(list(train['cat'+str(i)])+list(test['cat'+str(i)]))
    train['cat'+str(i)] = le.transform(train['cat'+str(i)])
    test['cat'+str(i)] = le.transform(test['cat'+str(i)])

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
X= train.drop('target',axis=1)
y = train['target']

let's use **train_test_split** to avoid overfitting:

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
X_train.shape,X_test.shape

checking for correlation b/w columns:

In [ ]:
X_train.corr()

In [ ]:
import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(14,14))
cor = X_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X_train,0.6)
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
X_train.drop('id',axis=1)
X_test.drop('id',axis=1)
X_train.drop(corr_features,axis=1)
X_test.drop(corr_features,axis=1)

training xgbclassifier model:

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=200,
                     learning_rate=0.2,
                     gamma=0.1,
                     max_depth=6)
model.fit(X_train,y_train)

In [ ]:
pred = model.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred)

predicting for **test** dataset and storing prediction into csv file:

In [ ]:
test_pred = model.predict_proba(test)[:,1]
test['target'] = test_pred
submission =test[['id','target']]
submission.to_csv('submission.csv',index=False)

well, i'm doing any hyper parameters tuning, i'll keep it simple.

thanks.